# **기본세팅**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 주요 라이브러리 설치
!pip install mxnet
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# seed 값 설정
torch.manual_seed(41)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import transformers
from transformers import AdamW

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#klue bert-base 토크나이저를 불러옵니다: https://huggingface.co/klue/bert-base
from transformers import AutoTokenizer

MODEL_TYPE ="klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_TYPE)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# device 설정
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


# **klue bert-base모델**

In [ ]:
# 데이터 불러오기
train = pd.read_csv("/content/drive/MyDrive/공모전/용재/문장유형분류 데이콘 데이터/train.csv")
test = pd.read_csv("/content/drive/MyDrive/공모전/용재/문장유형분류 데이콘 데이터/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/공모전/용재/문장유형분류 데이콘 데이터/sample_submission.csv")

In [ ]:
train

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


데이터전처리

In [ ]:
import re

for i in range(len(train['문장'])):
  train['문장'][i] = str(re.sub('\([^)]+\)', '',train['문장'][i] ))

In [ ]:
# train['문장'] = train['문장'].str.replace(" +", " ") # 다중 공백 제거
# train['문장'] = train['문장'].str.replace("《", "")
# train['문장'] = train['문장'].str.replace("》", "")
# train['문장'] = train['문장'].str.replace("＂", "")
# train['문장'] = train['문장'].str.replace("＇", "")

# **Label 인코딩**

In [ ]:
# label 인코딩
label_dict_type = {"사실형" : 0, "추론형" : 1, "대화형" : 2, '예측형' : 3}
label_dict_polarity = {"긍정" : 0, "부정" : 1, "미정" : 2}
label_dict_tense = {"과거" : 0, "현재" : 1, "미래" : 2}
label_dict_certainty = {"확실" : 0, "불확실" : 1}
#유형	극성	시제	확실성	

# **하이퍼파라미더 조절**

In [ ]:
# 모델 하이퍼파라미터
L_RATE = 3e-5 
MAX_LEN = 256 

BATCH_SIZE = 16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

In [ ]:
# 학습 횟수
NUM_EPOCHS=10


# **○여기서 원하는 라벨 학습○**




# **유형**

In [ ]:
train['label_num'] = [label_dict_type[i] for i in train.유형]
train.head()

,ID,문장,유형,극성,시제,확실성,label,label_num
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0


In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]

Dataloader 만들기

In [ ]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence = self.df_data.loc[index, '문장']

        encoded_dict = tokenizer.encode_plus(
                    sentence,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence = self.df_data.loc[index, '문장']


        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [ ]:
from transformers import AutoModel,BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 4, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)
#모델 로드
# model.load_state_dict(torch.load('/content/epoch:1_model.pt'))

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

Train(학습)

In [ ]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
# loss값 저장
loss_values = []

# 학습 시작
for epoch in tqdm( range(NUM_EPOCHS)):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)

    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        loss = outputs[0]
        
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값
        preds = outputs[1]


        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred, average='weighted')
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)
    print('F1 Score: ', val_f1)

    # 모델 저장
    torch.save(model.state_dict(), '/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_lr3e5_유형_epoch_{}.pt'.format(epoch+1))
    
    # 메모리 관리
    gc.collect()

  0%|          | 0/10 [00:00<?, ?it/s]


======== Epoch 1 / 10 ========
Training...
Train loss: 257.69134516455233

Validation...
Val loss: 50.58354886248708
Val acc:  0.9008763977032336
F1 Score:  0.8887917567881871


 10%|█         | 1/10 [10:21<1:33:09, 621.02s/it]


======== Epoch 2 / 10 ========
Training...
Train loss: 166.4536722006742

Validation...
Val loss: 64.96601173561066
Val acc:  0.899365367180417
F1 Score:  0.8965474370051939


 20%|██        | 2/10 [20:47<1:23:15, 624.46s/it]

...

 80%|████████  | 8/10 [1:23:27<20:53, 626.58s/it]


======== Epoch 9 / 10 ========
Training...
Train loss: 22.307657129869767

Validation...
Val loss: 179.51242288395952
Val acc:  0.8878815352070112
F1 Score:  0.8906337707155195


 90%|█████████ | 9/10 [1:33:54<10:26, 626.69s/it]


======== Epoch 10 / 10 ========
Training...
Train loss: 18.710480722173088

Validation...
Val loss: 158.5293436067368
Val acc:  0.898458748866727
F1 Score:  0.8958610316573382


100%|██████████| 10/10 [1:44:21<00:00, 626.17s/it]


# **극성**

In [ ]:
train['label_num'] = [label_dict_polarity[i] for i in train.극성]
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]

Dataloader 만들기

In [ ]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence = self.df_data.loc[index, '문장']

        encoded_dict = tokenizer.encode_plus(
                    sentence,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence = self.df_data.loc[index, '문장']


        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [ ]:
from transformers import AutoModel,BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)
#모델 로드
# model.load_state_dict(torch.load('/content/epoch:1_model.pt'))

In [ ]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

Train(학습)

In [ ]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
# loss값 저장
loss_values = []

# 학습 시작
for epoch in tqdm( range(NUM_EPOCHS)):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)

    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        loss = outputs[0]
        
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값
        preds = outputs[1]


        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred, average='weighted')
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)
    print('F1 Score: ', val_f1)

    # 모델 저장
    torch.save(model.state_dict(), '/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_극성_epoch_{}.pt'.format(epoch+1))
    
    # 메모리 관리
    gc.collect()

# **시제**

In [ ]:
train['label_num'] = [label_dict_tense[i] for i in train.시제]
train.head()

,ID,문장,유형,극성,시제,확실성,label,label_num
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,1
1,TRAIN_00001,이어 앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정이라며 그 이전이라도 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,2
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,1


In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]

Dataloader 만들기

In [ ]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence = self.df_data.loc[index, '문장']

        encoded_dict = tokenizer.encode_plus(
                    sentence,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence = self.df_data.loc[index, '문장']


        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [ ]:
from transformers import AutoModel,BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)
#모델 로드
# model.load_state_dict(torch.load('/content/epoch:1_model.pt'))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

Train(학습)

In [ ]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
# loss값 저장
loss_values = []

# 학습 시작
for epoch in tqdm( range(NUM_EPOCHS)):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)

    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        loss = outputs[0]
        
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값
        preds = outputs[1]


        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred, average='weighted')
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)
    print('F1 Score: ', val_f1)

    # 모델 저장
    torch.save(model.state_dict(), '/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_시제_epoch_{}.pt'.format(epoch+1))
    
    # 메모리 관리
    gc.collect()

# **확실성**

In [ ]:
train['label_num'] = [label_dict_certainty[i] for i in train.확실성]
train.head()

,ID,문장,유형,극성,시제,확실성,label,label_num
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0
1,TRAIN_00001,이어 앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정이라며 그 이전이라도 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0


In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]

Dataloader 만들기

In [ ]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence = self.df_data.loc[index, '문장']

        encoded_dict = tokenizer.encode_plus(
                    sentence,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence = self.df_data.loc[index, '문장']


        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [ ]:
from transformers import AutoModel,BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 2, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)
#모델 로드
# model.load_state_dict(torch.load('/content/epoch:1_model.pt'))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

Train(학습)

In [ ]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
# loss값 저장
loss_values = []

# 학습 시작
for epoch in tqdm( range(NUM_EPOCHS)):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)

    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        loss = outputs[0]
        
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값
        preds = outputs[1]


        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred, average='weighted')
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)
    print('F1 Score: ', val_f1)

    # 모델 저장
    torch.save(model.state_dict(), '/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_확실성_epoch_{}.pt'.format(epoch+1))
    
    # 메모리 관리
    gc.collect()

# **Inference**

In [ ]:
path_type='/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_유형_epoch_9.pt'
path_polarity='/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_극성_epoch_9.pt'
path_tense='/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_시제_epoch_9.pt'
path_certainty='/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_확실성_epoch_9.pt'

# **유형**

In [ ]:
train['label_num'] = [label_dict_type[i] for i in train.유형]

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
#test 예측에 사용
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )    
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
%%capture
from transformers import AutoModel,BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE,
    num_labels = 4
)
# model을 device위에 올림
model.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [ ]:
test_data = TestDataset(test)
MAX_LEN = 256 
BATCH_SIZE = 16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

# batch_size 만큼 데이터 분할
test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
model.load_state_dict(torch.load(path_type))
# model.load_state_dict(torch.load('/content/drive/MyDrive/공모전/용재/데이콘PT/klue_bert_확실성_epoch_0.pt'))
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 예측
for j, batch in enumerate(test_dataloader):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

In [ ]:
preds = np.argmax(stacked_preds, axis=1)
pred_type=[]
for i in preds:
  if i == 0:
    pred_type.append("사실형")
  elif i == 1:
    pred_type.append("추론형")
  elif i == 2:
    pred_type.append("대화형")
  elif i == 3:
    pred_type.append("예측형")

#**극성**

In [ ]:
train['label_num'] = [label_dict_polarity[i] for i in train.극성]

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
#test 예측에 사용
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )    
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
%%capture
from transformers import AutoModel,BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE,
    num_labels = 3
)
# model을 device위에 올림
model.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [ ]:
test_data = TestDataset(test)
MAX_LEN = 256 
BATCH_SIZE = 16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

# batch_size 만큼 데이터 분할
test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
model.load_state_dict(torch.load(path_polarity))
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 예측
for j, batch in enumerate(test_dataloader):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

In [ ]:
preds = np.argmax(stacked_preds, axis=1)
pred_polarity=[]
for i in preds:
  if i == 0:
    pred_polarity.append("긍정")
  elif i == 1:
    pred_polarity.append("부정")
  elif i == 2:
    pred_polarity.append("미정")
    # label_dict_polarity = {"긍정" : 0, "부정" : 1, "미정" : 2}

#**시제**

In [ ]:
train['label_num'] = [label_dict_tense[i] for i in train.시제]

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
#test 예측에 사용
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )    
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
%%capture
from transformers import AutoModel,BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE,
    num_labels = 3
)
# model을 device위에 올림
model.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [ ]:
test_data = TestDataset(test)
MAX_LEN = 256 
BATCH_SIZE = 16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

# batch_size 만큼 데이터 분할
test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
model.load_state_dict(torch.load(path_tense))
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 예측
for j, batch in enumerate(test_dataloader):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

In [ ]:
preds = np.argmax(stacked_preds, axis=1)
pred_tense=[]
for i in preds:
  if i == 0:
    pred_tense.append("과거")
  elif i == 1:
    pred_tense.append("현재")
  elif i == 2:
    pred_tense.append("미래")

#**확실성**

In [ ]:
train['label_num'] = [label_dict_certainty[i] for i in train.확실성]

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.2)

In [ ]:
#test 예측에 사용
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer.encode_plus(
                    sentence,          
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )    
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
%%capture
from transformers import AutoModel,BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE,
    num_labels = 2
)
# model을 device위에 올림
model.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [ ]:
test_data = TestDataset(test)
MAX_LEN = 256 
BATCH_SIZE = 16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

# batch_size 만큼 데이터 분할
test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)

In [ ]:
model.load_state_dict(torch.load(path_certainty))
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 예측
for j, batch in enumerate(test_dataloader):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

In [ ]:
preds = np.argmax(stacked_preds, axis=1)
pred_certainty=[]
for i in preds:
  if i == 0:
    pred_certainty.append("확실")
  elif i == 1:
    pred_certainty.append("불확실")

# **합쳐합쳐!!**

In [ ]:
predictions = []
for p_type, p_polarity, p_tense, p_certainty in zip(pred_type, pred_polarity, pred_tense, pred_certainty):
    predictions.append(p_type+'-'+p_polarity+'-'+p_tense+'-'+p_certainty)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/공모전/용재/문장유형분류 데이콘 데이터/sample_submission.csv')
submit['label'] = predictions

In [ ]:
submit

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실
...,...,...
7085,TEST_7085,사실형-긍정-현재-확실
7086,TEST_7086,추론형-긍정-현재-확실
7087,TEST_7087,사실형-긍정-미래-확실
7088,TEST_7088,추론형-긍정-미래-확실


In [ ]:
submit.to_csv('/content/klue_bert_epoch_10.csv', index=False) 